Usar a célula abaixo no [gist](https://gist.github.com/PauloGSC/d2844e133de645525a88ad38f5f0d22e). As instruções estão no próprio gist.

In [ ]:
from datetime import datetime, timedelta; print((datetime.now() - timedelta(hours=3)).strftime('%d/%m/%Y %H:%M:%S'))

# Importando utilitários do Jupyter



In [ ]:
import IPython
from IPython.display import Image

print(IPython.version_info)

# Removendo pasta *sample_data*

In [ ]:
%rm -rf sample_data

# Clonando o repositório

In [ ]:
%rm -rf yolov3
! git clone --depth 1 https://github.com/PauloGSC/yolov3.git
%cd yolov3

# Criando diretórios `data/samples` e `data/output`

In [ ]:
%mkdir -p data/samples/img data/samples/vid
%mkdir -p data/output/img data/output/vid

# Instalando dependências

In [ ]:
! pip install -r requirements.txt

# Obtendo pesos pré-treinados pela ultralytics

In [ ]:
from os.path import join
from models import *

pfx = 'weights/ultralytics'
weights = ['yolov3.pt']
for w in weights:
    attempt_download(join(pfx, w))

# Verificando GPU

In [ ]:
import torch
print(torch.cuda.get_device_properties(0) if torch.cuda.is_available else "cpu")

# Sobrescrevendo hiperparâmetros

In [ ]:
! > hyp-custom.txt # limpa o arquivo primeiro
! echo "giou 0.5" >> hyp-custom.txt # adiciona ao final
! echo "iou_t 0.4" >> hyp-custom.txt # adiciona ao final

! cat hyp-custom.txt

# Treinando

In [ ]:
! python train.py --cfg cfg/assistente-yolov3.cfg --weights weights/ultralytics/yolov3.pt --epochs 15 --batch-size 8 --freeze-layers

# Aparando _best.pt_

In [ ]:
from utils.utils import strip_optimizer; strip_optimizer()

# Testando

In [ ]:
! python test.py --cfg cfg/assistente-yolov3.cfg --weights weights/best.pt --batch-size 8

# Plotando resultados

In [ ]:
# ! python -c "from utils import utils; utils.plot_results()"
Image(filename='results.png', height=400)

# Inferência

In [ ]:
! python detect.py --source data/samples/img --output data/output/img --cfg cfg/assistente-yolov3.cfg --weights weights/best.pt

In [ ]:
! python detect.py --source data/samples/vid --output data/output/vid --cfg cfg/assistente-yolov3.cfg --weights weights/best.pt

# Plotar imagem

In [ ]:
Image(filename='data/output/img/dtc-teste-00.jpg', height=400)

# Zipando pesos, saídas e resultados

In [ ]:
! zip -r e3-8.zip data/output weights/best.pt results.png results.txt

# Removendo resquícios do último treinamento

In [ ]:
# não esquecer de antes fazer o download dos arquivos
%rm -rf *.zip weights/last.pt weights/best.pt train_batch* test_batch* results* data/*/*.npy data/*.shapes data/output/img/* data/output/vid/* runs/*

# Célula pra fazer qualquer outra coisa